In [ ]:
import random
import copy

# 1. 적합도 함수 정의
def fitness(perm):
    """
    적합도 함수: 28 - h
    h는 서로 대각선에서 공격하는 퀸 쌍의 수
    """
    h = 0
    for i in range(8):
        for j in range(i + 1, 8):
            # 대각선 공격 조건: |i - j| == |perm[i] - perm[j]|
            if abs(i - j) == abs(perm[i] - perm[j]):
                h += 1
    return 28 - h

# 2. 초기 개체군 생성
def generate_population(pop_size):
    """
    초기 개체군 생성: 무작위 순열로 구성
    pop_size: 개체군 크기
    """
    population = []
    for _ in range(pop_size):
        # 0~7의 무작위 순열 생성
        perm = list(range(8))
        random.shuffle(perm)
        population.append(perm)
    return population

# 3. 토너먼트 선택
def tournament_selection(population, tournament_size=3):
    """
    토너먼트 선택: 무작위로 tournament_size만큼 선택 후 가장 적합한 개체 반환
    """
    selected = random.sample(population, tournament_size)
    selected.sort(key=fitness, reverse=True)  # 적합도 기준 내림차순 정렬
    return selected[0]

# 4. 교차 (Partially Mapped Crossover, PMX)
def pmx_crossover(parent1, parent2):
    """
    PMX 교차: 순열을 유지하면서 교차
    """
    child = [-1] * 8
    # 교차 지점 선택
    start, end = sorted(random.sample(range(8), 2))
    
    # 부모1의 start~end 부분을 자식에게 복사
    child[start:end + 1] = parent1[start:end + 1]
    
    # 부모2에서 나머지 부분 채우기
    for i in range(start, end + 1):
        value = parent2[i]
        if value not in child:
            curr = value
            while True:
                pos = parent1.index(curr)
                if child[pos] == -1:
                    child[pos] = value
                    break
                curr = parent2[pos]
    
    # 나머지 -1인 부분 채우기
    for i in range(8):
        if child[i] == -1:
            child[i] = parent2[i]
    
    return child

# 5. 돌연변이 (Swap Mutation)
def mutate(perm, mutation_rate=0.01):
    """
    돌연변이: 낮은 확률로 두 위치를 교환
    """
    perm = perm.copy()
    if random.random() < mutation_rate:
        i, j = random.sample(range(8), 2)
        perm[i], perm[j] = perm[j], perm[i]
    return perm

# 6. 유전 알고리즘 메인 함수
def genetic_algorithm(pop_size=50, max_generations=1000, mutation_rate=0.01):
    """
    유전 알고리즘으로 8-Queens 문제 해결
    pop_size: 개체군 크기
    max_generations: 최대 세대 수
    mutation_rate: 돌연변이 확률
    """
    # 초기 개체군 생성
    population = generate_population(pop_size)
    
    for generation in range(max_generations):
        # 적합도 계산
        best_individual = max(population, key=fitness)
        best_fitness = fitness(best_individual)
        
        # 적합도 28이면 최적 해답 발견
        if best_fitness == 28:
            print(f"Solution found in generation {generation}!")
            print(f"Best individual: {best_individual}")
            print(f"Fitness: {best_fitness}")
            return best_individual
        
        # 새로운 개체군 생성
        new_population = []
        
        # 엘리트 보존: 가장 좋은 개체를 다음 세대에 직접 복사
        new_population.append(best_individual)
        
        # 나머지 개체 생성
        while len(new_population) < pop_size:
            # 부모 선택
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            
            # 교차
            child = pmx_crossover(parent1, parent2)
            
            # 돌연변이
            child = mutate(child, mutation_rate)
            
            new_population.append(child)
        
        population = new_population
    
    # 최대 세대에 도달했으나 해답을 찾지 못함
    best_individual = max(population, key=fitness)
    best_fitness = fitness(best_individual)
    print("No solution found within max generations.")
    print(f"Best individual: {best_individual}")
    print(f"Fitness: {best_fitness}")
    return best_individual

# 7. 결과 출력 및 체스판 시각화
def print_board(solution):
    """
    해답을 체스판에 시각화
    """
    board = [['.' for _ in range(8)] for _ in range(8)]
    for row, col in enumerate(solution):
        board[row][col] = 'Q'
    
    print("\nSolution Board:")
    for row in board:
        print(' '.join(row))

# 메인 실행
if __name__ == "__main__":
    # 유전 알고리즘 실행
    solution = genetic_algorithm(pop_size=50, max_generations=1000, mutation_rate=0.01)
    
    # 결과 출력
    print_board(solution)

: 